In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import collections
import socket
import struct
import os
import pickle
import time

In [26]:
ScalerPath = "./GeneratedData/Scaler.pickle"
ModelPath = "./GeneratedData/Model.pickle"
HeaderWithoutAccel = ["ApproachRate", 
                      "Rel_Pos_Magnitude", 
                      "SteerB", 
                      "A_Head_Center_Distance", "B_Head_Center_Distance", 
                      "Filtered_B_Head_Velocity_Total",
                      "A_Turn",
                      "B_Indicator",
                      "Centerline_Offset_B",
                      "RelativeRotation"]
outputHeader = ["AccelA"]
input_size = len(HeaderWithoutAccel)
hidden_size = 8
TargetTimeLength = 0.75 ## in seconds
Seq_Length = 13
output_size = len(outputHeader)

In [33]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, seq_length, num_layers=1):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length


        # GRU layer
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * seq_length, output_size)

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Reshape output to fit the fully connected layer
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [31]:
scaler=None
if os.path.exists(ScalerPath):
    with open(ScalerPath, 'rb') as file:
        scaler = pickle.load(file)
else:
    print ("Could NOT find Scaler")

In [34]:
net = GRUModel(len(HeaderWithoutAccel), hidden_size, output_size,Seq_Length)
net.load_state_dict(torch.load(ModelPath))
net.eval()
d = collections.deque(maxlen=Seq_Length)

In [36]:
# Initial Status
for i in range(10):
    d.append(np.ones(input_size)+i)
scaled_features = scaler.transform(np.array(d))
net(torch.Tensor([scaled_features]))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (13,) + inhomogeneous part.

In [5]:
UDP_IP = "127.0.0.1"
txPort = 8000
rxPort = 8001
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, rxPort))

In [7]:
while True:
    try:
        data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
       
        if(int(len(data)/4)!=7):
            print("Didnt get enoughb bytes, continuing!",int(len(data)/4))
            continue
        array = struct.unpack('fffffff', data) 

        d.append(np.array(array))
        scaled_features = scaler.transform(np.array(d))
        val = net(torch.Tensor([scaled_features]))
        floatlist=[val]
        #print(array,val)
        buf = struct.pack('%sf' % len(floatlist), *floatlist)
        sock.sendto(buf, (UDP_IP, txPort))
    except BlockingIOError:
        time.sleep(1/19)
        